In [ ]:
import os
from lib.train_AI_lib import *
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
os.chdir('..')
print(os.getcwd())

In [ ]:
def genBBoxImages(dictPath='saved/splitData', inPath='data/working-wheat-data/train', oPath='saved/bboxMasked'):
    try: os.makedirs(oPath)  # Make the requested oPath
    except FileExistsError: None
    except: print("error creating folder {}".format(oPath)); return(0)
    else: None
    
    trainData = torch.load(dictPath+"/trainData")
    valData   = torch.load(dictPath+"/valData"  )
    testData  = torch.load(dictPath+"/testData" )
    trainData.update(valData); trainData.update(testData)
    imgPath   = inPath+'/'
    keyList   = list(trainData.keys())

    for i, imgName in enumerate(keyList):
        oimg  = cv2.imread(imgPath+imgName)
        bbox = trainData[imgName]
        mask = createMask(bbox, np.shape(oimg)[:-1])
        if i%100==0: print("Converted {:.2f}%".format(100*i/len(keyList)))
        np.save(oPath+'/'+imgName.split('.jpg')[0], mask)

In [ ]:
#genBBoxImages()

In [ ]:
class autEncA(nn.Module):
    def __init__(self, name):
        super(autEncA, self).__init__()
        self.name = name
        self.encoder = nn.Sequential( # like the Composition layer you built
            nn.Conv2d(3, 5, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(5, 10, 3, stride=2, padding=1),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(10, 5, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(5, 3, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
calcNoParam(autEncA(''))

In [ ]:
class autEncB(nn.Module):
    def __init__(self, name):
        super(autEncB, self).__init__()
        self.name = name
        self.encoder = nn.Sequential( # like the Composition layer you built
            nn.Conv2d(3, 5, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(5, 10, 4, stride=2, padding=1),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(10, 5, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(5, 3, 4, stride=2, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
calcNoParam(autEncB(''))

In [ ]:
class autEncC(nn.Module):
    def __init__(self, name):
        super(autEncC, self).__init__()
        self.name = name
        self.encoder = nn.Sequential( # like the Composition layer you built
            nn.Conv2d(3, 5, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(5, 7, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(7, 10, 4, stride=2, padding=1)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(10, 7, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(7, 5, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(5, 3, 4, stride=2, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
calcNoParam(autEncC(''))

In [ ]:
class autEncD(nn.Module):  # Ratio of Input Size to code size = 12.8
    def __init__(self, name):
        super(autEncD, self).__init__()
        self.name = name
        self.encoder = nn.Sequential( # like the Composition layer you built
            nn.Conv2d(3, 6, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(6, 10, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(10, 15, 4, stride=2, padding=1)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(15, 10, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(10, 6, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(6, 3, 4, stride=2, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
calcNoParam(autEncD(''))

In [ ]:
class autEncE(nn.Module):  # Ratio of inputsize to code size 3.15
    def __init__(self, name):
        super(autEncE, self).__init__()
        self.name = name
        self.encoder = nn.Sequential( # like the Composition layer you built
            nn.Conv2d(3, 6, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(6, 10, 4, stride=2, padding=1),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(10, 6, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(6, 3, 4, stride=2, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
calcNoParam(autEncE(''))

In [ ]:
batchsize=64; epoch=15; lr=0.001
net = autEncB('autEncB'); net.cuda()
trainLoader, valLoader, testLoader = loadData(batchsize, tempPath='temp/auto', mode='auto', preCalc=0, altArg={'compPath':'saved/bboxMasked'})
_ = trainNet(net, [trainLoader, valLoader], batchsize, epoch, lr, trainType='AutoEnc')

In [ ]:
batchsize=64; epoch=15; lr=0.001
net = autEncC('autEncC'); net.cuda()
trainLoader, valLoader, testLoader = loadData(batchsize, tempPath='temp/auto', mode='auto', preCalc=0, altArg={'compPath':'saved/bboxMasked'})
_ = trainNet(net, [trainLoader, valLoader], batchsize, epoch, lr, trainType='AutoEnc')

In [ ]:
batchsize=64; epoch=10; lr=0.001
net = autEncD('autEncD'); net.cuda()
trainLoader, valLoader, testLoader = loadData(batchsize, tempPath='temp/auto', mode='auto', preCalc=0, altArg={'compPath':'saved/bboxMasked'})
_ = trainNet(net, [trainLoader, valLoader], batchsize, epoch, lr, trainType='AutoEnc')

In [ ]:
batchsize=64; epoch=10; lr=0.001
net = autEncE('autEncE'); net.cuda()
trainLoader, valLoader, testLoader = loadData(batchsize, tempPath='temp/auto', mode='auto', preCalc=0, altArg={'compPath':'saved/bboxMasked'})
_ = trainNet(net, [trainLoader, valLoader], batchsize, epoch, lr, trainType='AutoEnc')

Looking at the results of NetD and NetE, there dosent seem to be a signficant difference
Both reached lowest rMSE of around 0.238. However, there is merit to the thought that when these networks are put through semi-supervised environments, they will perform differently
due to the difference no of core variables

Based on this paper: https://arxiv.org/pdf/1511.00561.pdfhttps://arxiv.org/pdf/1511.00561.pdf
Changing the loading process so that the comp image is a 1-layer of binary classification
